In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers==4.28.0
!pip install datasets evaluate
!pip install konlpy

# Importing Package

In [ ]:
import collections
import numpy as np
import string
import pandas as pd

import logging
import json
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertTokenizerFast,
    AlbertModel,
    DataCollatorWithPadding,
    PreTrainedTokenizerFast,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

# Loading Data + Preprocessing

In [ ]:
RANDOM_SEED = 80

# get stopwords list
stopwords_file_path = '/content/drive/MyDrive/QIA2023_phase1/data/stopwords.txt'
lines = open(stopwords_file_path, "r")

filter_list = []
for word in lines:
  filter_list.append(word.replace("\n", "").replace("\ufeff", ""))
filter_list.append(".")
filter_list.append(",")

from konlpy.tag import Okt
from konlpy.utils import pprint

okt = Okt()

# remove stopwords and punctuation
def preprocess_str(text):
  text = okt.morphs(text, norm=True) # not lemma
  text = [word for word in text if word not in filter_list]
  result = text[0]
  for word in text[1:]:
    result = result + " " + word
  return result

question_df = pd.read_excel('/content/drive/MyDrive/QIA2023_phase1/data/Question.xlsx')['Question']
# question_df = question_df.apply(preprocess_str)


# Remove <> and apply preprocess_str
def preprocess_ans(word):
  lst = word.split(">")
  return lst[1] + " [SEP] " + lst[0][1:]

def preprocess_data(data_df):
  data_df["Answer"] = data_df["Answer"].apply(preprocess_ans)
  data_df["Q_number"] = data_df["Q_number"].apply(lambda idx: question_df[idx - 1])
  data_df['Answer'] = data_df["Q_number"] + " [SEP] " + data_df['Answer'] + " [SEP] " + data_df['Age'].astype(str) + " [SEP] " + data_df['Gender'].astype(str)
  return data_df

In [ ]:
# load datasets
df = pd.read_csv('/content/drive/MyDrive/QIA2023_phase1/data/train_data_s1_v1.csv', encoding='cp949', index_col=0)
df = preprocess_data(df)

In [ ]:
df.tail()

In [ ]:
# split train and test dataframe
train_df_list = []
val_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:45])
    val_df_list.append(df[df['User_ID']==idx][45:])

In [ ]:
train_df = pd.concat(train_df_list, ignore_index=True)
val_df = pd.concat(val_df_list, ignore_index=True)

In [ ]:
train_df = train_df.sample(frac=1, random_state=RANDOM_SEED)
train_df.tail()

In [ ]:
val_df = val_df.sample(frac=1, random_state=RANDOM_SEED)
val_df.tail()

# Loading pretrained model checkpoint

In [ ]:
# I = 0, E = 1
# S = 0, N = 1
# J = 0, P = 1
# T = 0, F = 1
label_class = [['I', 'E'], ['S', 'N'], ['T', 'F'], ['J', 'P']]

labels = ["ESTJ", "ENTJ", "ESFJ", "ENFJ", "ISTJ", "ISFJ", "INTJ", "INFJ", "ESTP", "ESFP", "ENTP", "ENFP", "ISTP", "ISFP", "INTP", "INFP"]

id2label = dict()
label2id = dict()

for i, label in enumerate(labels):
  label2id[label] = i
  id2label[i] = label

In [ ]:
def get_dataset():
  train_dataset = {
      "text": [],
      "label": []
  }

  for index, row in train_df.iterrows():
      train_dataset["text"].append(row["Answer"])
      train_dataset["label"].append(label2id[row["MBTI"]])

  val_dataset = {
      "text": [],
      "label": []
  }

  for index, row in val_df.iterrows():
      val_dataset["text"].append(row["Answer"])
      val_dataset["label"].append(label2id[row["MBTI"]])

  return datasets.DatasetDict({"train":datasets.Dataset.from_dict(train_dataset),"test":datasets.Dataset.from_dict(val_dataset)})

# TensorFlow model

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from transformers import AutoModelForMaskedLM

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from transformers import TFBertModel

AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 3
BATCH_SIZE = 8
MAX_LEN = 512

In [ ]:
from sklearn.model_selection import train_test_split

# Hugging Face model

In [ ]:
# model_checkpoint = "klue/roberta-large"
# model_checkpoint = "klue/bert-base"
model_checkpoint = "beomi/kcbert-large"

config    = AutoConfig.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model     = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=16, id2label=id2label, label2id=label2id
)

In [ ]:
model

In [ ]:
dataset = get_dataset()
dataset

In [ ]:
example = dataset['train'][0]
example

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation = True)

In [ ]:
tokenized_ds = dataset.map(preprocess_function, batched = True)

# Metrics

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Class Training Arguments + Trainer

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir = "checkpoints",
    overwrite_output_dir = 'True',
    learning_rate = 1e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 50,
    weight_decay = 0.0,
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 500,
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

In [ ]:
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

# Training

In [ ]:
trainer.train()
# trainer.save_model("checkpoints")

# Deploy Model

In [ ]:
from tqdm import tqdm
import torch, gc

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/QIA2023_phase1/data/hackathon_test_for_user.csv', encoding='cp949')
test_df = preprocess_data(test_df)

In [ ]:
# checkpoint_idx = 18000
# model_checkpoint = f"/content/drive/MyDrive/QIA2023_phase1/checkpoint/checkpoint-{checkpoint_idx}"

In [ ]:
model_checkpoint = '/content/checkpoints/checkpoint-500'

config    = AutoConfig.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model     = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=16, id2label=id2label, label2id=label2id
).to(device)

In [ ]:
texts = []

for index, row in test_df.iterrows():
    texts.append(row['Answer'])

In [ ]:
final_logits = {
    'I/E': [],
    'S/N': [],
    'T/F': [],
    'J/P': []
}

for i in tqdm(range(0, len(texts), 16)):
    current_batch_size = min(16, len(texts) - i)

    inputs = texts[i: i + current_batch_size]
    inputs = tokenizer(inputs)
    inputs = data_collator(inputs).to(device)

    logits = model(**inputs).logits
    logits = torch.nn.Softmax(dim = 1)(logits)
    logits = torch.permute(logits, (1, 0)).cpu().data
    one_logits = torch.zeros(4, current_batch_size)

    gc.collect()
    torch.cuda.empty_cache()

    for label in labels:
        if ('E' in label):  one_logits[0] += logits[label2id[label]]
        if ('N' in label):  one_logits[1] += logits[label2id[label]]
        if ('F' in label):  one_logits[2] += logits[label2id[label]]
        if ('P' in label):  one_logits[3] += logits[label2id[label]]
    
    final_logits['I/E'] += one_logits[0].tolist()
    final_logits['S/N'] += one_logits[1].tolist()
    final_logits['T/F'] += one_logits[2].tolist()
    final_logits['J/P'] += one_logits[3].tolist()

In [ ]:
result = pd.DataFrame(final_logits)
result.index += 1
result.to_csv('/content/drive/MyDrive/to_submit/Results/Phase1/kcbert_00.csv', index_label="idx")

In [ ]:
# test = pd.DataFrame({
#     'I/E': [MBTI[0] for MBTI in MBTIs], 
#     'S/N': [MBTI[1] for MBTI in MBTIs], 
#     'T/F': [MBTI[2] for MBTI in MBTIs], 
#     'J/P': [MBTI[3] for MBTI in MBTIs], 
# })
# test.index += 1
# test.to_csv(f'/content/drive/MyDrive/QIA2023_phase1/result/single-flow-1/result-{checkpoint_idx}.csv', index_label="idx")

In [ ]:
# %cp -av /content/checkpoints/checkpoint-500 /content/drive/MyDrive/QIA2023_phase1/checkpoint